In [1]:
# Libraries:
import pysindy as ps
import numpy as np
import matplotlib.pyplot as plt
import dill
import sys
import os

from sklearn.metrics import mean_squared_error
from sklearn.metrics import root_mean_squared_error
from sklearn.linear_model import Lasso

sys.path.append("/home/mattg/D_CODE") # Necessario al momento per far girare toolbox
from D_CODE.run_simulation import run as run_SRT
from D_CODE.run_simulation_vi import run as run_DCODE

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..'))) # include parent directory in the path
from toolbox.auxiliary_functions import set_param_freq
from data import equations
from data import SINDy_data

/home/mattg/miniforge3/envs/SimbSINDy_env/lib/python3.11/site-packages/pysindy/__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound


In [2]:
# Procedura ideale:

# Step 1: Dati HD
# - Genera i dati gompertz -> z_dot = a * z * log ( b*z )
# - proiezione con i polimoni di Legiandre -> N=128

# Step 2: AE+SINDy with Polynomial CFL: degree 3
# Project onto the latent space
# SR: D-CODE with 5 generations and {+,x,-,-1,log} function set

# Step 3:
# Integrate the building block in final_library
# SINDy on final library

#### Step 1: High-Dimentional data Generation

Consider the one-dimentional Gompertz Model:
$$ \dot{z}(t) = - a z(t) \log\!\bigl(b z(t)\bigr), \qquad a,b \in \mathbb{R}^{+} $$

In [3]:
# Select ODE & settings:
ode_name = 'GompertzODE' # help="name of the ode", type=str
ode_param = '1.5,1.5' # help="parameters of the ode (default: None)", type=str, default=None
freq = 10 # help="sampling frequency", type=float, default=10
dt = 1 / freq
n_sample = 20 # help="number of trajectories", type=int, default=100
noise_ratio = 0.01 # help="noise level (default 0)", type=float, default=0.
dim_x = 1 # help="dimentionality of the problem", type=int
dim_k = 0 # help="dimentionality of the parameterizaiton discovery", type=int
ode_param, freq = set_param_freq(ode_param, freq)
ode = equations.get_ode(ode_name, ode_param)
time_vector = np.arange(0, ode.T + dt, dt)

In [5]:
# generate data:
Z_list, dZ_list, param_list, feature_names = SINDy_data.SINDy_data(ode_name=ode_name, 
                                                                   ode_param=ode_param,
                                                                   freq=freq,
                                                                   n_sample=n_sample,
                                                                   noise_ratio=noise_ratio,
                                                                   dim_x=dim_x,
                                                                   dim_k=dim_k
                                                                   )

# ensure existance conditions:
Z_list = SINDy_data.existence_conditions(Z_list, ode.init_low, n_variables=dim_x)

print(np.shape(Z_list), np.shape(dZ_list), np.shape(param_list), feature_names)

(20, 160, 1) (20, 160, 1) (0,) ['X0']


The high-dimentional dataset is obtained by concatenating multiple instances of a nonlinear combination of the true Gompertz dynamics:

$$
\mathbf{x}(t) = \mathbf{u_1} z(t) + \mathbf{u_2} z(t)^3 
$$

where $\mathbf{u_1}, \mathbf{u_2} \in \mathbb{R}^{128}$ are the first two Legendre polynomials.

In [ ]:
# Modes Generation with Legendre Polynomials
from numpy.polynomial.legendre import legval

# Number of points and grid
N = 128
x = np.linspace(-1, 1, N)

# Generation
u1 = legval(x, [0, 1]).reshape(1,-1)       # coeff [P0, P1] -> reshape (1,N)
u2 = legval(x, [0, 0, 1]).reshape(1,-1)    # coeff [P0, P1, P2] -> reshape(1,N)

print(u1.shape, u2.shape)

In [ ]:
# Projeciton on the modes:
X_list = []
for z in Z_list:
    x = np.array(z) @ u1 + np.array(z)**3 @ u2
    X_list.append(x)
print(np.shape(X_list))

#### Step 2: AE+SINDy training and projection